In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import os
import time
import re

In [ ]:
os.chdir('..')
if not os.path.exists("download"):
    os.makedirs("download")
os.chdir('download')

In [ ]:
chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : os.getcwd(),
        'download.prompt_for_download': False,
        'download.extensions_to_open': 'xml',
        'safebrowsing.enabled': True}
chrome_options.add_experimental_option('prefs', prefs)
chrome_options.add_argument('--start-maximized')
driver = webdriver.Chrome(service = ChromeService(ChromeDriverManager().install()), options=chrome_options)

In [ ]:
driver.get('https://www.myexperiment.org/workflows?filter=TYPE_ID%28%221%22%29&num=100')

In [ ]:
def erase_files():
    files = os.listdir()
    if(len(files) > 0):
        for file in files:
            try:
                os.remove(file)
                return 1
            except:
                time.sleep(0.5)
                erase_files()
                return 0

In [ ]:
def download_file(element, seconds, workflow_type):
    erase_files()
    try:
        element.click()
        time.sleep(seconds)
        workflow_number = re.findall(r'\d+',element.get_attribute("href"))[0]
        file = os.listdir()[0]
        if file.endswith('download'):
            print(f'{file} -> {workflow_number} DOWNLOAD ERROR')
            download_file(element, seconds)
            return 0
        else:
            file_type = re.findall(r'(?<=\.)\s*(\w+)', file)[-1]
            os.rename(file,f"..//workflows_myExperiment//workflows_{workflow_type}//{workflow_number}.{file_type}") #Taverna 1
            print(f'{file} -> {workflow_number} SAVED')
            return 1
    except FileExistsError:
            print(f'{file} -> {workflow_number} DOUBLED')
            return 1

In [ ]:
while(True):
    try:
        downloads = driver.find_elements(By.XPATH,"//a[contains(@href,'download')]")
        for download in downloads:
            download_file(download, 5, "taverna2")
        next = driver.find_element(By.XPATH,"//a[contains(@class,'next_page')]")
        next.click()
        time.sleep(5)
    except NoSuchElementException:
        print("Finished")
        break